## Data projet abritel

In [17]:
from IPython.display import display
import json
import pandas as pd
pd.options.display.max_columns = None # Display all columns of a dataframe
pd.options.display.max_rows = 700
from pprint import pprint
import re
import requests
from bs4 import BeautifulSoup

import time
import os
from selenium import webdriver
from selenium.webdriver.common. by import By

#vérifier à la fin qu'on a bien utiliser ça
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [18]:
#Add following options before initializing the webdriver

# chromeOptions = webdriver.ChromeOptions()
# chromeOptions.add_argument("--headless")
# chromeOptions.add_argument("--remote-debugging-port=9222")
#chromeOptions.add_argument('--no-sandbox')

path="C:/Users/32498/Downloads/ECON2206-Data-Management-2023-main/chromedriver_win32"
driver = webdriver.Chrome(executable_path=os.path.join(path, 'chromedriver'))
driver.get('https://www.abritel.fr/search/keywords:ardennes-belges-r%C3%A9gion-de-la-wallonie-belgique/arrival:2023-08-14/departure:2023-08-27/minNightlyPrice/0?filterByTotalPrice=true&petIncluded=false&ssr=true&adultsCount=6&childrenCount=0')
time.sleep(3)

#reject cookies
reject_all_button = driver.find_element(By.XPATH,"//button[contains(@class, 'osano-cm-denyAll')]")
reject_all_button.click()

time.sleep(2)


# 1. Web scraping part

In [15]:
base_url='https://www.abritel.fr/search/keywords:ardennes-belges-r%C3%A9gion-de-la-wallonie-belgique/page:{}/arrival:2023-08-14/departure:2023-08-27/minNightlyPrice/0?filterByTotalPrice=true&petIncluded=false&ssr=true&adultsCount=6'

num_pages = 4

all_names = []
price_night=[]
price_total=[]
rating_info=[]
types=[]
features_list=[]
premium_list = []

for page_num in range(1, num_pages+1):
    link = base_url.format(page_num)
    driver.get(link)
    time.sleep(2) # attendre quelques secondes pour que la page se charge complètement
    
    for i in range(1, 22):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight / 22 * {});".format(i))
        time.sleep(1)
    
    # Extraire les noms de la page en cours
    names = driver.find_elements(By.CSS_SELECTOR, 'h2.HitInfo__headline')
    name_list = [name.text for name in names]
    all_names.extend(name_list)
    
    prices_night = driver.find_elements(By.CSS_SELECTOR, 'span.DualPrice__secondary')
    night_list=[price_night.text for price_night in prices_night]
    price_night.extend(night_list)
    
    prices_total = driver.find_elements(By.CSS_SELECTOR, 'span.DualPrice__primary')
    total_list=[price_total.text for price_total in prices_total]
    price_total.extend(total_list)
    
    type_houses=driver.find_elements(By.CSS_SELECTOR,'div.HitInfo__content div.HitInfo__type-place-details span')
    type_list=[type_house.text for type_house in type_houses]
    types.extend(type_list)
    
   # premium_hosts = driver.find_elements(By.CSS_SELECTOR, 'span.HitInfo__labels_item')
   # premium_hosts=0
   # if premium_hosts:
    #    premium_host = [host.text for host in premium_hosts]
   #    premium.extend(premium_host) # ajouter "Hote premium" au début de la liste
    annonces = driver.find_elements(By.CSS_SELECTOR, 'div.HitCarousel__badge')
    for annonce in annonces:
        premium = annonce.find_elements(By.CSS_SELECTOR, 'span.HitInfo__labels')
    if premium:
        premium_list.append(1)
    else:
        premium_list.append(0)
    
    infos=driver.find_elements(By.CSS_SELECTOR,'div.HitInfo__starRating')
    for info in infos:
        rating_num = '0'
        rating_count = '0 appréciation'
        rating_status = 'rien'
    
        rating_nums = info.find_elements(By.CSS_SELECTOR, 'span.Rating__label')
        if rating_nums:
            rating_num = rating_nums[0].text

        rating_counts = info.find_elements(By.CSS_SELECTOR, 'span.Rating__count')
        if rating_counts:
            rating_count = rating_counts[0].text

        rating_3 = info.find_elements(By.CSS_SELECTOR, 'span.HitInfo__ranked')
        if rating_3:
            rating_status = rating_3[0].text
        
        rating_info.append([rating_num, rating_count, rating_status])

    for rating in rating_info:
        rating=pd.DataFrame(rating_info, columns=["Rating", "Appreciations","Type of annoucement"])                   
    
    elements = driver.find_elements(By.CSS_SELECTOR, "div.HitInfo__room-beds-details")
    for element in elements:
        span_elements = element.find_elements(By.TAG_NAME, "span")
        personnes = re.sub("[^0-9]", "", span_elements[0].text) if span_elements else '0'
        chambres = re.sub("[^0-9]", "", span_elements[1].text) if len(span_elements) > 1 else '0'
        lits = re.sub("[^0-9]", "", span_elements[2].text) if len(span_elements) > 2 else "0"
        features_list.append([personnes,chambres,lits])

features = pd.DataFrame(features_list, columns=["Number of Guests", "Number of Bedrooms", "Number of Beds"])

driver.quit()

In [8]:
rating

,Rating,Appreciations,Type of annoucement
0,5.0,(37 appréciations),Annonce professionnelle
1,5.0,(14 appréciations),rien
2,4.9,(14 appréciations),Particulier
3,0,(0 appréciation),rien
4,0,(0 appréciation),rien
5,4.0,(86 appréciations),Annonce professionnelle
6,4.6,(10 appréciations),Particulier
7,4.2,(23 appréciations),Particulier
8,4.7,(23 appréciations),Particulier
9,0,(0 appréciation),rien


In [16]:
premium

[]

In [9]:
data = {
    'Name': all_names,
    'Price per Night': price_night,
    'Total Price': price_total,
    'Type': types,
    'Number of Guests': [f[0] for f in features_list],
    'Number of Bedrooms': [f[1] for f in features_list],
    'Number of Beds': [f[2] for f in features_list]
}
# Créer le dataframe à partir du dictionnaire
df1 = pd.DataFrame(data)
df1
# Afficher le dataframe

ValueError: All arrays must be of the same length

In [35]:
features

,Number of Guests,Number of Bedrooms,Number of Beds
0,8,3,4
1,9,3,7
2,8,4,5
3,8,3,5
4,8,1,7
5,8,3,7
6,6,2,4
7,12,5,12
8,6,2,5
9,8,1,1


In [36]:
df=pd.concat([df1,rating,features], axis=1)
df.set_index('Name', inplace=True)
df

,Price per Night,Total Price,Type,Number of Guests,Number of Bedrooms,Number of Beds,Rating,Appreciations,Type of annoucement,Number of Guests,Number of Bedrooms,Number of Beds
Name,,,,,,,,,,,,
Vieille maison de vacances à Couvin près de la rivière,107 €/nuit,1 386 €montant total,Maison,8,3,4,5.0,(37 appréciations),Annonce professionnelle,8,3,4
Charmante maison de vacances au coeur de l'Ardenne à Sainte-Marie-Chevigny,129 €/nuit,1 680 €montant total,Maison,9,3,7,5.0,(14 appréciations),rien,9,3,7
"Nouveau gite au calme avec magnifique vue - campagne - 4 chambres, 2 sdb",333 €/nuit,4 325 €montant total,Maison,8,4,5,4.9,(14 appréciations),Particulier,8,4,5
"Maison à la frontière de Namur, du Luxembourg et de la France",140 €/nuit,1 825 €montant total,Maison,8,3,5,0,(0 appréciation),rien,8,3,5
Maison de vacances cozy à Bièvre dans un quartier charmant,133 €/nuit,1 733 €montant total,Maison,8,1,7,0,(0 appréciation),rien,8,1,7
Gîte confortable à Paliseul avec jardin,103 €/nuit,1 335 €montant total,Cottage,8,3,7,4.9,(18 appréciations),Annonce professionnelle,8,3,7
"Maison de vacances cosy, four, nature environnante",97 €/nuit,1 261 €montant total,Maison,6,2,4,4.0,(86 appréciations),Annonce professionnelle,6,2,4
"Ferme rénovée avec sauna, bain à bulles et 5 chambres près de Bouillon",234 €/nuit,3 046 €montant total,Maison,12,5,12,4.6,(10 appréciations),Particulier,12,5,12
"Chalet confortable en bois avec micro-ondes, situé en forêt",88 €/nuit,1 145 €montant total,Chalet,6,2,5,3.8,(116 appréciations),Annonce professionnelle,6,2,5


In [78]:
locations = driver.find_elements(By.XPATH, "//div[@class='media-flex__content']/a")

# Boucle pour visiter chaque page de location et récupérer les données souhaitées
for location in locations:
    # Clique sur le lien de la location
    location_url = location.get_attribute('href')
    driver.get(location_url)
    time.sleep(3)

    # récupère la localisation
    localization = driver.find_element(By.XPATH, "//div[@class='listing-headline']/div[@class='listing-headline-wrapper']/h1")
    localization_text = localization.text

    print(localization_text)

    driver.back()

In [3]:
locations = driver.find_elements(By.CLASS_NAME,"SimpleImageCarousel")

# Boucle pour visiter chaque page de location et récupérer les données souhaitées
for location in locations:
    # Clique sur le lien de la location
    #location_page=driver.execute_script("arguments[0].click();", location)
    location_url=f"https://www.abritel.fr/search/keywords:ardennes-belges-r%C3%A9gion-de-la-wallonie-belgique/arrival:2023-08-14/departure:2023-08-27/minNightlyPrice/0?filterByTotalPrice=true&petIncluded=false&ssr=true&adultsCount=6&childrenCount=0{location.find_element('a')[0].get_attribute('href')}"
    driver.get(location_url)
    time.sleep(3)
    
    #récupère la localisation
    localization = driver.find_element(By.CSS_SELECTOR, 'a.u-hover-pointer')
    localization_text = localization.text

    print(localization_text)

    driver.back()

InvalidArgumentException: Message: invalid argument: invalid locator
  (Session info: chrome=111.0.5563.149)


# 2. Visualization

In [ ]:
import matplotlib.pyplot as plt

                         
x = df["Appreciations"]
y = df["Price per Night"]

x_sorted, y_sorted = zip(*sorted(zip(x, y)))
# Create the scatter plot
plt.scatter(x_sorted, y_sorted)

In [ ]:
# Group the data by rating and compute the mean of the prices for each rating
mean_prices = df.groupby('Appreciations')['Price per Night'].mean()

# Print the mean prices for each rating
print(mean_prices)

In [ ]:
import numpy as np


colormap = plt.cm.Set1
num_colors = len(df["Price Range"].unique())
colors = [colormap(i) for i in np.linspace(0, 0.9, num_colors)]
fig, ax = plt.subplots()
for i, price_range in enumerate(df["Price Range"].unique()):
    df_subset = df[df["Price Range"] == price_range]
    ax.scatter(df_subset["Appreciations"], df_subset["Price per Night"], color=colors[i], label=price_range)
ax.legend()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Group the data by "Price Range" and create a list of price data for each group
groups = []
for name, group in df.groupby("Price Range"):
    groups.append(group["Price per Night"])

# Create the violin plot
sns.violinplot(data=groups)

# Set the x-axis label
plt.xlabel("Price Range")

# Set the y-axis label
plt.ylabel("Price per Night")

# Show the 
plt.show()

# 3. Machine learning

In [1]:
df_numerical = df.select_dtypes(include='number')
features= df_numerical.drop(columns=['Price per night','Total price'])#input
features

NameError: name 'df' is not defined

In [ ]:
import seaborn as sns
df_numerical = df.select_dtypes(include='number')
features= df_numerical.drop(columns=['Price per night','Total price'])#input

sns.set(rc={'figure.figsize':(8.5,5)})
correlation_matrix = features.corr().round(2)#pq round (2)?
sns.heatmap(correlation_matrix) #annot=True
#heatmat c'est une fonction de seaborn
plt.show()